**Students who have not submitted “Assignment_101”**

In [ ]:
table_name='Gold_assignment_101_defaulters'
df=spark.sql("""
with students as 
    (
select 
    student_id,
    full_name,
    email_address 
from silver_students 
    ),
submissions as (
select 
    student_id 
from silver_submissions
where assignment_id='Assignment_101'
    ) 
select 
    s.student_id as student_id,
    full_name as full_name,
    email_address as email_address  
from students s 
left join submissions sub
on s.student_id=sub.student_id
WHERE sub.student_id IS NULL

""")
df.write.mode("overwrite").saveAsTable(table_name)

StatementMeta(, 4f9f0d01-c8d6-43a3-a1a6-e400c2d9f610, 3, Finished, Available, Finished)

**Overdue assignment count per student**

In [ ]:
table_name='Gold_overdue_assignments'
df=spark.sql("""
    SELECT 
        ss.student_id as student_id,
        count(*) as Assignment_count
    FROM silver_students_submissions ss
    JOIN silver_assignment_courses ac
        ON ss.assignment_id = ac.assignment_id
    where ss.submission_date > ac.due_date
    group by ss.student_id
""")
df.write.mode("overWrite").saveAsTable(table_name)

StatementMeta(, 4f9f0d01-c8d6-43a3-a1a6-e400c2d9f610, 9, Finished, Available, Finished)

**Submission rate per course (as a %)**

In [ ]:
table_name='Gold_submissions_ratio'
df=spark.sql("""
WITH CourseStats AS (
    SELECT 
        c.course_id, 
        c.course_name,
        COUNT(DISTINCT a.assignment_id) AS total_assignments,
        COUNT(DISTINCT sub.submission_id) AS total_submissions
    FROM silver_courses c
    LEFT JOIN silver_assingments a ON c.course_id = a.course_id
    LEFT JOIN silver_submissions sub ON a.assignment_id = sub.assignment_id
    GROUP BY c.course_id, c.course_name
),
StudentCount AS (
    SELECT COUNT(DISTINCT student_id) AS total_students FROM silver_students
)

SELECT 
    cs.course_id,
    cs.course_name,
    ROUND(
        (cs.total_submissions * 1.0 / NULLIF(cs.total_assignments * sc.total_students, 0)) * 100,
        2
    ) AS submission_rate_percent
FROM CourseStats cs
CROSS JOIN StudentCount sc;
""")

df.write.mode("overWrite").saveAsTable(table_name)


StatementMeta(, 755f86d9-c2e9-4395-9b35-47ae4c52247c, 5, Finished, Available, Finished)